# Projeto 1 - Ciência dos Dados

Nome: Layne Silva

Nome: Lidia Domingos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [200]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lidia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [201]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lidia\Desktop\Ciência dos Dados\Projeto1\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [202]:
filename = 'XIAOMI.xlsx'

In [203]:
train = pd.read_excel(filename)
train.head(5)

,TREINAMENTO,CLASSIFICAÇÃO
0,quem aqui tb pronuncia xiaomi xaiomi,0
1,xiaomi mi true airdots 2s earphone(aliexpress)...,0
2,@felpsflip carreguei meu xiaomi uma vez na vid...,1
3,"#promo #\n$133.99 ~ 111,85 € ✂️cupão/cupom/cup...",0
4,"#promo #\n$299.99 ~ 250,41 € ✂️cupão/cupom/cup...",0


In [204]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0
3,comprei mais capinha pro xiaomi https://t.co/7...,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0


In [205]:
test.iloc[:,0:1]

,Teste
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon..."
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ..."
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p..."
3,comprei mais capinha pro xiaomi https://t.co/7...
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...
...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi..."
255,@mslaryhill tenho um haylou gt1 pro q é da mes...
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...
257,@inutializador @viniport0 @2kjuramento @pessoa...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

*Escolhida a empresa XIAOMI, classificamos como tweets relevantes toda e qualquer citação (seja ela positiva ou negativa) dos internautas falando sobre o produto. Tweets considerados como irrelevantes seriam tweets não relacionados diretamente ao produto.*

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [329]:
#Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.
#Correção de espaços entre palavras e/ou emojis.
#Proposta de outras limpezas/transformações que não afetem a qualidade da informação.

import re 
from nltk.tokenize.casual import TweetTokenizer #separa os emojis como palavras.

def cleanup(text):
    #import string
    punctuation = '[!-@.~[\]:"?;\),("/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '',text_subbed)
    t = TweetTokenizer()
    text_subbed = t.tokenize(text_subbed) 
    return text_subbed

def RemoveStopWords(list_of_words):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    lista_filtrada = []
    for word in list_of_words:
        if word not in stopwords:
            lista_filtrada.append(word)
    return lista_filtrada

def Suavizacao_Laplace(palavra1, palavra2, palavra3):
    y = (1 + palavra1)/(len(palavra2) + palavra3)  
    return y

def MultiplyList(List) : 
    result = 1
    for x in List: 
         result = result * x  
    return result


In [330]:
#JUNTANDO TODAS AS PALAVRAS DOS TWEETS RELEVANTES DO TREINAMENTO
train_relevante = train.loc[train["CLASSIFICAÇÃO"] == 1, :]
train_relevante_filtrada = ' '.join(train_relevante.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train["CLASSIFICAÇÃO"] == 0, :]
train_irrelevante_filtrada = ' '.join(train_irrelevante.TREINAMENTO) 


In [331]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
relevante = cleanup(train_relevante_filtrada.lower())
palavras_relevantes = RemoveStopWords(relevante)

irrelevante = cleanup(train_irrelevante_filtrada.lower())
palavras_irrelevantes = RemoveStopWords(irrelevantes)

#JUNTANDO OS CONJUNTOS
palavras_total = palavras_relevantes + palavras_irrelevantes


In [332]:
#TRANSFORMANDO EM PD.SERIES
series_total = pd.Series(palavras_total)
series_train_relevante = pd.Series(palavras_relevantes)
series_train_irrelevante = pd.Series(palavras_irrelevantes)

In [333]:
#FREQUÊNCIAS RELATIVAS
palavras_total_relativas = series_train_global.value_counts(True)
palavras_relevante_relativas = series_train_relevante.value_counts(True)
palavras_irrelevante_relativas = series_train_irrelevante.value_counts(True)


In [334]:
#FREQUÊNCIAS ABSOLUTAS
palavras_relevante_absolutas = series_train_relevante.value_counts()
palavras_irrelevante_absolutas = series_train_irrelevante.value_counts()
palavras_total_absolutas = series_train_global.value_counts()
palavras_total_absolutas

xiaomi             194
bom                 30
celular             26
q                   24
redmi               18
                  ... 
chamado              2
press_chris          2
desconsiderar        2
história             2
luluvalcarenghi      2
Length: 638, dtype: int64

In [335]:
palavras_total_absolutas["xiaomi"]

194

**EVENTOS**

 * $R$: comentários relevantes
 * $R^c$: comentários irrelevantes
 * $C$: comentário recebido

**PROBABILIDADES**

 * $P(R)$ : probabilidade do comentário ser relevante ;
 * $P(R^c)$: probabilidade do comentário não ser relevante, ou seja, irrelevante ;
 * $P(C)$ : probabilidade de cada palavra ocorrer na língua portuguesa ;
 * $P(C|R)$ : probabilidade do comentário existir dado o conjunto de relevantes ;
 * $P(C|R^c)$: probabilidade do comentário existir dado o conjunto de irrelevantes ; 
 * $P(R|C)$: probabilidade do comentário ser relevante dado a frase ;
 * $P(R^c|C)$: probabilidade do comentário ser irrelevante dado a frase;


**CLASSIFICADOR**



In [428]:
def funcao_classificadora(test):
    #separando os comentários
    
    test['classificação_nova'] = 'NaN'
    
    comentarios = test.iloc[:,0:1]
    comentarios_juntos = ' '.join(comentarios.iloc[:,0]) 
    palavras_limpas_comentario = RemoveStopWords(cleanup(comentarios_juntos.lower()))
    palavras_serie_comentario = pd.Series(palavras_limpas_comentario)
    palavras_comentarios = palavras_serie_comentario.value_counts()
     
    P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
    P_Rc = 1 - P_R
    
    for tweet in list(test.Teste):
        tweet_limpo = RemoveStopWords(cleanup(tweet.lower()))
        lista_de_probabilidades1 = []
        lista_de_probabilidades2 = []
        for palavra in tweet_limpo:
            if palavra in palavras_relevante_absolutas:
                Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_total_absolutas,palavras_relevante_absolutas.sum())
                lista_de_probabilidades1.append(Laplace)
            if palavra == 'xiaomi':
                continue
            else:
                Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_relevante_absolutas.sum())
                lista_de_probabilidades1.append(Laplace)
            P_C_dado_R = MultiplyList(lista_de_probabilidades1)

        for palavra in tweet_limpo:
            if palavra in palavras_irrelevante_absolutas:
                Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
                lista_de_probabilidades2.append(Laplace)
            if palavra == 'xiaomi':
                continue
            else:
                Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
                lista_de_probabilidades2.append(Laplace)
            P_C_dado_Rc = MultiplyList(lista_de_probabilidades2)
        
        P_R_dado_C = P_R * P_C_dado_R

        P_Rc_dado_C = P_Rc * P_C_dado_Rc            
        
        if P_R_dado_C > P_Rc_dado_C:
            resultado = 1
        else:
            resultado = 0
            
        test['classificação_nova'] = resultado

    return test
    

In [429]:
funcao_classificadora(test)


,Teste,CLASSIFICAÇÃO,classificação_nova
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0,0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0,0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0,0
3,comprei mais capinha pro xiaomi https://t.co/7...,0,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0,0
...,...,...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi...",0,0
255,@mslaryhill tenho um haylou gt1 pro q é da mes...,1,0
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...,0,0
257,@inutializador @viniport0 @2kjuramento @pessoa...,0,0


In [419]:
test['classificação_nova'] = 'NaN'
lista = []
lista1 = []
comentarios = test.iloc[:,0:1]
comentarios_juntos = ' '.join(comentarios.iloc[:,0]) 
palavras_limpas_comentario = RemoveStopWords(cleanup(comentarios_juntos.lower()))
palavras_serie_comentario = pd.Series(palavras_limpas_comentario)
palavras_comentarios = palavras_serie_comentario.value_counts()

# P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
# P_Rc = 1 - P_R
P_R = 0.6
p_Rc = 1 - P_R

for tweet in list(test.Teste):
    tweet_limpo = RemoveStopWords(cleanup(tweet.lower()))
    lista_de_probabilidades1 = []
    lista_de_probabilidades2 = []
    for palavra in tweet_limpo:
        if palavra in palavras_relevante_absolutas:
            Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_total_absolutas,palavras_relevante_absolutas.sum())
            lista_de_probabilidades1.append(Laplace)
        if palavra == 'xiaomi':
            continue
        else:
            Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_relevante_absolutas.sum())
            lista_de_probabilidades1.append(Laplace)
        P_C_dado_R = MultiplyList(lista_de_probabilidades1)

    for palavra in tweet_limpo:
        if palavra in palavras_irrelevante_absolutas:
            Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
            lista_de_probabilidades2.append(Laplace)
        if palavra == 'xiaomi':
            continue
        else:
            Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
            lista_de_probabilidades2.append(Laplace)
        P_C_dado_Rc = MultiplyList(lista_de_probabilidades2)

    P_R_dado_C = P_R * P_C_dado_R

    P_Rc_dado_C = P_Rc * P_C_dado_Rc            

    if P_R_dado_C > P_Rc_dado_C:
        resultado = 1
    else:
        resultado = 0
    lista.append(P_R_dado_C)
    lista1.append(P_Rc_dado_C)
    test['classificação_nova'] = resultado


In [420]:
lista

[4.2978412492418354e-39,
 2.794999350398812e-62,
 6.823252767296342e-35,
 6.021604895296727e-16,
 8.806450206765595e-42,
 6.936814245875934e-24,
 1.5058229754436606e-126,
 7.913629106555294e-85,
 2.466647994221869e-32,
 1.238294020731558e-34,
 2.5473203077421148e-43,
 8.700157907198149e-47,
 1.3209675310148396e-42,
 5.740818175766401e-115,
 5.549451396700746e-23,
 6.043287585843581e-31,
 2.840837339745353e-58,
 3.5090636892365874e-45,
 1.184484366825342e-28,
 8.813222499385373e-21,
 4.755483111653422e-41,
 1.1522979610067797e-130,
 1.6847537697027997e-36,
 1.3152166148447689e-49,
 1.4688704165642285e-20,
 2.196541134217814e-109,
 9.309505351739345e-85,
 2.282607348077699e-50,
 1.893891559830235e-58,
 3.965950124723418e-20,
 9.602586266578316e-121,
 7.140419084842857e-12,
 1.1990778910728397e-22,
 2.1937133936612204e-28,
 5.052997376205058e-54,
 1.8111595790584293e-58,
 2.8871324667634247e-55,
 2.41917398594476e-08,
 2.6917105804609815e-88,
 1.1426705269065411e-119,
 5.861947486337248e-

In [421]:
lista1

[3.5815343743681964e-39,
 2.3291661253323434e-62,
 5.686043972746952e-35,
 5.018004079413939e-16,
 7.338708505637997e-42,
 5.780678538229945e-24,
 1.254852479536384e-126,
 6.594690922129412e-85,
 2.055539995184891e-32,
 1.031911683942965e-34,
 2.122766923118429e-43,
 7.250131589331791e-47,
 1.1008062758456997e-42,
 4.784015146472002e-115,
 4.624542830583955e-23,
 5.036072988202984e-31,
 2.367364449787794e-58,
 2.9242197410304896e-45,
 9.870703056877851e-29,
 7.344352082821144e-21,
 3.962902593044518e-41,
 9.60248300838983e-131,
 1.4039614747523333e-36,
 1.0960138457039742e-49,
 1.2240586804701906e-20,
 1.8304509451815117e-109,
 7.757921126449455e-85,
 1.902172790064749e-50,
 1.5782429665251957e-58,
 3.3049584372695154e-20,
 8.002155222148597e-121,
 5.950349237369047e-12,
 9.992315758940332e-23,
 1.8280944947176837e-28,
 4.210831146837549e-54,
 1.509299649215358e-58,
 2.405943722302854e-55,
 2.0159783216206335e-08,
 2.2430921503841514e-88,
 9.522254390887843e-120,
 4.8849562386143736e-8

In [262]:
P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
P_R

0.5

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [430]:
funcao_classificadora(test)

,Teste,CLASSIFICAÇÃO,classificação_nova
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0,0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0,0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0,0
3,comprei mais capinha pro xiaomi https://t.co/7...,0,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0,0
...,...,...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi...",0,0
255,@mslaryhill tenho um haylou gt1 pro q é da mes...,1,0
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...,0,0
257,@inutializador @viniport0 @2kjuramento @pessoa...,0,0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**